In [13]:
from __future__ import print_function
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd
import torch.optim as optim
import torch.backends.cudnn as cudnn
import argparse
import sys
import time
import datetime 
import csv
import configparser
import platform
from torchvision import datasets, transforms
from torch.autograd import Variable
from random import shuffle

In [126]:
transform = transforms.Compose([transforms.Grayscale(),transforms.ToTensor(),])
train_data_path = './horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
loader_train = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True, num_workers=1)
test_data_path = './horse-or-human/validation'
testset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
loader_test = torch.utils.data.DataLoader(valset, batch_size=10, shuffle=True, num_workers=1)

In [84]:
class Linear(nn.Module):

    def __init__(self, num_classes=2):

        super(Linear, self).__init__()

        self.number_class   = num_classes

        _size_image     = 100* 100
        _num1           = 50
        _num2           = 50
        
        self.fc1        = nn.Linear(_size_image, _num1, bias=True)
        self.fc2        = nn.Linear(_num1, _num2, bias=True)
        self.fc3        = nn.Linear(_num2, num_classes, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.ReLU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.ReLU(True))
        self.fc_layer3  = nn.Sequential(self.fc3, nn.ReLU(True))
        
        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2, self.fc_layer3)
        
        self._initialize_weight()        
        
    def _initialize_weight(self):        
            
        for m in self.modules():
            if isinstance(m, nn.Linear):
                n = m.in_features 
                m.weight.data.uniform_(- 1.0 / math.sqrt(n), 1.0 / math.sqrt(n))
                if m.bias is not None:
                    m.bias.data.zero_()

    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

In [101]:
num_classes = 2

In [85]:
model = Linear(num_classes=num_classes)

In [141]:
optimizer   = optim.SGD(model.parameters(),lr = 0.01)
objective   = nn.CrossEntropyLoss()

def train():

    # print('train the model at given epoch')

    loss_train          = []

    model.train()

    for idx_batch, (data, target) in enumerate(loader_train):

#         if bCuda:
        
#             data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        optimizer.zero_grad()

        output  = model(data)
        loss    = objective(output, target)

        loss.backward()
        optimizer.step()

        loss_train_batch    = loss.item() / len(data)
        loss_train.append(loss_train_batch)
        
    loss_train_mean     = np.mean(loss_train)
    loss_train_std      = np.std(loss_train)
    return {'loss_train_mean': loss_train_mean, 'loss_train_std': loss_train_std}

In [142]:
def test():

    # print('test the model at given epoch')

    accuracy_test   = []
    loss_test       = 0
    correct         = 0

    model.eval()

    for idx_batch, (data, target) in enumerate(loader_test):


        data, target    = Variable(data), Variable(target)

        output  = model(data)
        loss    = objective(output, target)

        loss_test   += loss.item()
        pred        = output.data.max(1)[1]
        correct     += pred.eq(target.data.view_as(pred)).cpu().sum()

    loss_test       = loss_test / len(loader_test.dataset)
    accuracy_test   = 100. * float(correct) / len(loader_test.dataset)
    return {'loss_test': loss_test, 'accuracy_test': accuracy_test}

In [ ]:
epoch = 10
loss_train_mean = [0 for i in range(epoch)]
loss_train_std = [0 for i in range(epoch)]
loss_test=[0 for i in range(epoch)]
accuracy_test=[0 for i in range(epoch)]
for e in range(epoch):
    print("epoch",e)
        
    result_train    = train()
    result_test     = test()

    loss_train_mean[e]  = result_train['loss_train_mean']
    loss_train_std[e]   = result_train['loss_train_std']
    loss_test[e]        = result_test['loss_test']
    accuracy_test[e]    = result_test['accuracy_test']

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
